In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date

from scipy.signal import butter,lfilter

from statsmodels.graphics.tsaplots import plot_predict

from statsmodels.tsa.arima.model import ARIMA
import arch

In [3]:
DATA_PATH = os.path.join(os.path.abspath('../../'), 'db', 'data')
TERTIARY_PATH = os.path.join(DATA_PATH, 'tertiary')
wpath = os.path.join(DATA_PATH, 'merge')

In [6]:
# Curency indexes dataframe
raw = pd.read_csv(os.path.join(wpath, os.path.basename(TERTIARY_PATH), 'logs_.csv'), parse_dates=['Unnamed: 0'], index_col='Unnamed: 0')
raw.index = pd.to_datetime(raw.index)

today = datetime.now()
hour = datetime( today.year, today.month, today.day, today.hour)
# today = date.today()
today
max=datetime(2022, 8, 1)
max = result = pd.Timestamp(max,tz='UTC')
max

Timestamp('2022-08-01 00:00:00+0000', tz='UTC')

In [7]:
df = raw[raw.index < max]

In [8]:
# Filter requirements.
T = df.shape[0]         # Sample Period
fs = 30.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,      slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency
order = 2       # sin wave can be approx represented as quadratic
n = int(T * fs) # total number of samples

def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

In [13]:
df = pd.DataFrame(butter_lowpass_filter(df, cutoff, fs, order), index=df.index, columns=df.columns)
split = int(len(df.index)/30)       
print(split)
train = df[:-split]
test = df[-split:]
# train.index = pd.to_datetime(train.index).to_period('H').to_timestamp().values
# test.index = pd.to_datetime(test.index).to_period('H').to_timestamp().values
print(f'\n\n{df.shape}\n{type(df.index)}\n{df.index[-1]}\n\n{train.shape}\n{train.index}\n\n{test.shape}\n{test.index}')

3840
(115221, 8)
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
2022-07-31 23:00:00+00:00

(111381, 8)

DatetimeIndex(['2005-01-02 19:00:00+00:00', '2005-01-02 20:00:00+00:00',
               '2005-01-02 21:00:00+00:00', '2005-01-02 22:00:00+00:00',
               '2005-01-02 23:00:00+00:00', '2005-01-03 00:00:00+00:00',
               '2005-01-03 01:00:00+00:00', '2005-01-03 02:00:00+00:00',
               '2005-01-03 03:00:00+00:00', '2005-01-03 04:00:00+00:00',
               ...
               '2021-12-23 10:00:00+00:00', '2021-12-23 11:00:00+00:00',
               '2021-12-23 12:00:00+00:00', '2021-12-23 13:00:00+00:00',
               '2021-12-23 14:00:00+00:00', '2021-12-23 15:00:00+00:00',
               '2021-12-23 16:00:00+00:00', '2021-12-23 17:00:00+00:00',
               '2021-12-23 18:00:00+00:00', '2021-12-23 19:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=111381, freq=None)

(3840, 8)

DatetimeIndex(['2021-12-23 20:00:00+00:00', '2021-12-23 21

## a)  Find p & q for each currency fro the last 10000 values of the trainset

{'AUD': (1, 0, 0), 'CAD': (2, 0, 1), 'CHF': (2, 0, 1), 'EUR': (2, 0, 1), 'GBP': (2, 0, 1), 'JPY': (2, 0, 1), 'NZD': (2, 0, 1), 'USD': (2, 0, 1)}

In [15]:
import warnings
from statsmodels.tsa.stattools import arma_order_select_ic

# parameters = {'AUD': [1,2],
#           'EUR': [1,2]...}
train_for_params = train[:-10000]

params = {}
with warnings.catch_warnings():
    warnings.simplefilter('once', category=Warning)
    for c in list(train_for_params.columns):
        res = arma_order_select_ic(train[c].values, max_ar=4, max_ma=2, ic='aic')
        # print(f'{c} mle retvals: {res.mle_retvals}')
        params[c] = (res.aic_min_order[0], 0, res.aic_min_order[1])
print(params)

/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check ml

{'AUD': (1, 0, 0), 'CAD': (2, 0, 1), 'CHF': (2, 0, 1), 'EUR': (2, 0, 1), 'GBP': (2, 0, 1), 'JPY': (2, 0, 1), 'NZD': (2, 0, 1), 'USD': (2, 0, 1)}


In [16]:
from termcolor import colored

arima_forc_dict = {'next_hour_index': []}

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=Warning)
    vol_dict = {'next_hour_index': []}
    for column in list(train_for_params.columns):

        arima = ARIMA(train_for_params[column], order=params[column])
        arima_fit = arima.fit()
        forecast_results = arima_fit.get_forecast(1, alpha=0.05) # get_forecast( ) define the length of predictions
        
        arima_forc_dict['next_hour_index'] = list(forecast_results.predicted_mean.index) # hourly indexes we predict
        arima_forc_dict[column] = list(forecast_results.predicted_mean) # forecast values
        
        # Use residuals in the Garch    
        garch = arch.arch_model(arima_forc_dict[column], p=1, q=1)
        garch_fitted = garch.fit()
        # Use GARCH to predict the residual
        garch_forecast = garch_fitted.forecast(horizon=1,reindex=False)
        predicted_et = garch_forecast.mean['h.1'].iloc[-1]
    
        # Combine both models' output: yt = mu + et
        volatility = arima_forc_dict[column] + predicted_et
        # Put them into dict
        vol_dict['next_hour_index'] = list(forecast_results.predicted_mean.index)
        vol_dict[column] = volatility
        print(colored(f'\n \n Predicted volatility for {column} currency is: {volatility}','red'))   
    
    vol_df = pd.DataFrame.from_dict(vol_dict).set_index('next_hour_index')

Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for AUD currency is: [-8.50405322e-06]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for CAD currency is: [2.99580845e-06]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for CHF currency is: [1.22591979e-05]


...

/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for NZD currency is: [0.00013252]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for USD currency is: [0.0002071]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


In [61]:
# perfomance = vol_predicted - vol_actual
perf = pd.DataFrame(vol_df-test.iloc[0]).reset_index(drop=True)   #, columns=vol_df.columns).set_index=test.iloc[0]
perf['next_hour_index'] = test.index[0]
perf.set_index('next_hour_index')

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
next_hour_index,,,,,,,,
2021-12-23 20:00:00+00:00,-0.000008,0.000009,0.000036,0.00008,0.00011,0.000076,-0.000059,-0.0003


In [57]:
test.index[0]

Timestamp('2021-12-23 20:00:00+0000', tz='UTC')

In [63]:
# Loop over all the test values
volatilities = {'next_hour_index': []}

with warnings.catch_warnings():
    warnings.simplefilter('ignore', category=Warning)
    for row in list(test.index):
        arima_forc_dict = {'next_hour_index': []}
        
        volatilities['next_hour_index'] = [row]
        
        vol_dict = {'next_hour_index': []}
        for column in list(train_for_params.columns):

            arima = ARIMA(train_for_params[column], order=params[column])
            arima_fit = arima.fit()
            forecast_results = arima_fit.get_forecast(1, alpha=0.05) # get_forecast( ) define the length of predictions
            
            arima_forc_dict['next_hour_index'] = list(forecast_results.predicted_mean.index) # hourly indexes we predict
            arima_forc_dict[column] = list(forecast_results.predicted_mean) # forecast values
            
            # Use residuals in the Garch    
            garch = arch.arch_model(arima_forc_dict[column], p=1, q=1)
            garch_fitted = garch.fit()
            # Use GARCH to predict the residual
            garch_forecast = garch_fitted.forecast(horizon=1,reindex=False)
            predicted_et = garch_forecast.mean['h.1'].iloc[-1]
        
            # Combine both models' output: yt = mu + et
            volatility = arima_forc_dict[column] + predicted_et
            # Put them into dict
            vol_dict['next_hour_index'] = list(forecast_results.predicted_mean.index)
            vol_dict[column] = volatility
            print(colored(f'\n \n Predicted volatility for {column} currency is: {volatility}','red'))   
            
            # One row of prediction in dict format
            volatilities[column] = [volatility]

Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for AUD currency is: [-8.50405322e-06]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for CAD currency is: [2.99580845e-06]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for CHF currency is: [1.22591979e-05]


...

Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for NZD currency is: [0.00013252]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


Iteration:      1,   Func. Count:      4,   Neg. LLF: nan
Inequality constraints incompatible    (Exit mode 4)
            Current function value: nan
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1

 
 Predicted volatility for USD currency is: [0.0002071]


/Users/pawel/.pyenv/versions/3.8.12/envs/rex-env/lib/python3.8/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


KeyboardInterrupt: 

In [64]:
volatilities

{'next_hour_index': [Timestamp('2021-12-24 05:00:00+0000', tz='UTC')],
 'AUD': [array([-8.50405322e-06])],
 'CAD': [array([2.99580845e-06])],
 'CHF': [array([1.22591979e-05])],
 'EUR': [array([3.08521888e-05])],
 'GBP': [array([5.64713156e-05])],
 'JPY': [array([8.76412152e-05])],
 'NZD': [array([0.00013252])],
 'USD': [array([0.0002071])]}

In [65]:
vol_df = pd.DataFrame.from_dict(volatilities).set_index('next_hour_index')

In [66]:
vol_df

,AUD,CAD,CHF,EUR,GBP,JPY,NZD,USD
next_hour_index,,,,,,,,
2021-12-24 05:00:00+00:00,[-8.504053217740633e-06],[2.99580844700799e-06],[1.2259197913438067e-05],[3.085218875846146e-05],[5.6471315631051435e-05],[8.764121518365675e-05],[0.00013251678038494544],[0.00020710201536950336]
